In [1]:
%%bash
pip install "monai-weekly[all]"
pip install scikit-image
pip install vtk
pip install huggingface_hub -U

In [2]:
import os
import tempfile
import numpy as np
import sys
import subprocess

# Install huggingface_hub if not available
try:
    import huggingface_hub
except ImportError:
    print("Installing huggingface_hub...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "huggingface_hub"])
    import huggingface_hub
    print("huggingface_hub installed successfully!")

import vtk
import vtkmodules

from monai.config import print_config
from monai.bundle.scripts import create_workflow, download
from monai.transforms import LoadImaged, SaveImage, Compose, BorderPadd, SqueezeDimd

print_config()

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


MONAI version: 1.6.dev2543
Numpy version: 2.3.4
Pytorch version: 2.9.0+cu128
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: bc5f3d1c1abae1389cd0e3d1a0bf3ff56e7f394d
MONAI __file__: /localhome/<username>/.conda/envs/monai/lib/python3.13/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.4
Nibabel version: 5.3.2
scikit-image version: 0.25.2
scipy version: 1.16.2
Pillow version: 12.0.0
Tensorboard version: 2.20.0
gdown version: 5.2.0
TorchVision version: 0.24.0+cu128
tqdm version: 4.67.1
lmdb version: 1.7.5
psutil version: 7.1.1
pandas version: 2.3.3
einops version: 0.8.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 3.5.1
pynrrd version: 1.1.3
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
# directory = os.environ.get("MONAI_DATA_DIRECTORY")
directory = "/localhome/local-tranminhq/datasets/maisi"
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/localhome/local-tranminhq/datasets/maisi


In [4]:
download(name="maisi_ct_generative", bundle_dir=root_dir)

2025-10-27 14:13:20,922 - INFO - --- input summary of monai.bundle.scripts.download ---
2025-10-27 14:13:20,923 - INFO - > name: 'maisi_ct_generative'
2025-10-27 14:13:20,924 - INFO - > bundle_dir: '/localhome/local-tranminhq/datasets/maisi'
2025-10-27 14:13:20,926 - INFO - > source: 'monaihosting'
2025-10-27 14:13:20,927 - INFO - > remove_prefix: 'monai_'
2025-10-27 14:13:20,928 - INFO - > progress: True
2025-10-27 14:13:20,928 - INFO - ---




Fetching 35 files: 100%|██████████| 35/35 [00:00<00:00, 1282.53it/s]


In [5]:
# 17 groupings that cover 101 segments/regions out of 140
labels = {
    "Liver": 1,
    "Spleen": 3,
    "Pancreas": 4,
    "Heart": 115,
    "Body": 200,
    "Gallbladder": 10,
    "Stomach": 12,
    "Small_bowel": 19,
    "Colon": 62,
    "Kidney": {"right_kidney": 5, "left_kidney": 14},
    "Veins": {
        "aorta": 6,
        "inferior_vena_cava": 7,
        "portal_vein_and_splenic_vein": 17,
        "left_iliac_artery": 58,
        "right_iliac_artery": 59,
        "left_iliac_vena": 60,
        "right_iliac_vena": 61,
        "pulmonary_vein": 119,
        "left_subclavian_artery": 123,
        "right_subclavian_artery": 124,
        "superior_vena_cava": 125,
        "brachiocephalic_trunk": 109,
        "left_brachiocephalic_vein": 110,
        "right_brachiocephalic_vein": 111,
        "left_common_carotid_artery": 112,
        "right_common_carotid_artery": 113,
    },
    "Lungs": {
        "left_lung_upper_lobe": 28,
        "left_lung_lower_lobe": 29,
        "right_lung_upper_lobe": 30,
        "right_lung_middle_lobe": 31,
        "right_lung_lower_lobe": 32,
    },
    "Spine": {
        "vertebrae_L6": 131,
        "vertebrae_L5": 33,
        "vertebrae_L4": 34,
        "vertebrae_L3": 35,
        "vertebrae_L2": 36,
        "vertebrae_L1": 37,
        "vertebrae_T12": 38,
        "vertebrae_T11": 39,
        "vertebrae_T10": 40,
        "vertebrae_T9": 41,
        "vertebrae_T8": 42,
        "vertebrae_T7": 43,
        "vertebrae_T6": 44,
        "vertebrae_T5": 45,
        "vertebrae_T4": 46,
        "vertebrae_T3": 47,
        "vertebrae_T2": 48,
        "vertebrae_T1": 49,
        "vertebrae_C7": 50,
        "vertebrae_C6": 51,
        "vertebrae_C5": 52,
        "vertebrae_C4": 53,
        "vertebrae_C3": 54,
        "vertebrae_C2": 55,
        "vertebrae_C1": 56,
        "sacrum": 97,
        "vertebrae_S1": 127,
    },
    "Ribs": {
        "left_rib_1": 63,
        "left_rib_2": 64,
        "left_rib_3": 65,
        "left_rib_4": 66,
        "left_rib_5": 67,
        "left_rib_6": 68,
        "left_rib_7": 69,
        "left_rib_8": 70,
        "left_rib_9": 71,
        "left_rib_10": 72,
        "left_rib_11": 73,
        "left_rib_12": 74,
        "right_rib_1": 75,
        "right_rib_2": 76,
        "right_rib_3": 77,
        "right_rib_4": 78,
        "right_rib_5": 79,
        "right_rib_6": 80,
        "right_rib_7": 81,
        "right_rib_8": 82,
        "right_rib_9": 83,
        "right_rib_10": 84,
        "right_rib_11": 85,
        "right_rib_12": 86,
        "costal_cartilages": 114,
        "sternum": 122,
    },
    "Shoulders": {"left_scapula": 89, "right_scapula": 90, "left_clavicula": 91, "right_clavicula": 92},
    "Hips": {"left_hip": 95, "right_hip": 96},
    "Back_muscles": {
        "left_gluteus_maximus": 98,
        "right_gluteus_maximus": 99,
        "left_gluteus_medius": 100,
        "right_gluteus_medius": 101,
        "left_gluteus_minimus": 102,
        "right_gluteus_minimus": 103,
        "left_autochthon": 104,
        "right_autochthon": 105,
        "left_iliopsoas": 106,
        "right_iliopsoas": 107,
    },
}


In [6]:
from re import L
from tkinter import Label


bundle_root = os.path.join(root_dir, "maisi_ct_generative")
override = {
    "output_size_xy": 256,
    "output_size_z": 256,
    "spacing_xy": 1.5,
    "spacing_z": 1.5,
    "autoencoder_def#num_splits": 16,
    "mask_generation_autoencoder_def#num_splits": 16,
    "body_region": ["chest"],
    "anatomy_list": ["left lung upper lobe", "left lung lower lobe", "right lung upper lobe", "right lung middle lobe", "right lung lower lobe"],
}
workflow = create_workflow(
    config_file=os.path.join(bundle_root, "configs/inference.json"),
    workflow_type="inference",
    bundle_root=bundle_root,
    **override,
)


2025-10-27 14:13:21,261 - INFO - Setting logging properties based on config: /localhome/local-tranminhq/datasets/maisi/maisi_ct_generative/configs/logging.conf.
2025-10-27 14:13:21,289 - INFO - --- input summary of monai.bundle.scripts.run ---
2025-10-27 14:13:21,290 - INFO - > workflow_type: 'inference'
2025-10-27 14:13:21,291 - INFO - > bundle_root: '/localhome/local-tranminhq/datasets/maisi/maisi_ct_generative'
2025-10-27 14:13:21,293 - INFO - > output_size_xy: 256
2025-10-27 14:13:21,294 - INFO - > output_size_z: 256
2025-10-27 14:13:21,296 - INFO - > spacing_xy: 1.5
2025-10-27 14:13:21,297 - INFO - > spacing_z: 1.5
2025-10-27 14:13:21,298 - INFO - > autoencoder_def#num_splits: 16
2025-10-27 14:13:21,299 - INFO - > mask_generation_autoencoder_def#num_splits: 16
2025-10-27 14:13:21,301 - INFO - > body_region: ['chest']
2025-10-27 14:13:21,303 - INFO - > anatomy_list: ['left lung upper lobe',
 'left lung lower lobe',
 'right lung upper lobe',
 'right lung middle lobe',
 'right lung l

Detected deprecated name 'optional_packages_version' in configuration file, replacing with 'required_packages_version'.


In [7]:
workflow.run()

2025-10-27 14:13:25,273 - INFO - 'dst' model updated: 180 of 231 variables.
2025-10-27 14:13:27,724 - root - INFO - `controllable_anatomy_size` is empty.
We will synthesize based on `anatomy_list`: (['left lung upper lobe', 'left lung lower lobe', 'right lung upper lobe', 'right lung middle lobe', 'right lung lower lobe']).
2025-10-27 14:13:27,725 - root - INFO - The generate results will have voxel size to be [1.5, 1.5, 1.5] mm, volume size to be [256, 256, 256].
2025-10-27 14:13:27,739 - root - INFO - LDM sampler initialized.
2025-10-27 14:13:27,786 - root - INFO - Resample mask file to get desired output size and spacing


monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.


2025-10-27 14:13:28,168 - root - INFO - Resampling mask to target shape and spacing
2025-10-27 14:13:28,173 - root - INFO - Resize Spacing: [tensor(1.5000, dtype=torch.float64), tensor(1.5000, dtype=torch.float64), tensor(1.5645, dtype=torch.float64)] -> [1.5, 1.5, 1.5]
2025-10-27 14:13:28,174 - root - INFO - Output size: [256, 256, 256] -> [256, 256, 256]
2025-10-27 14:13:29,613 - root - INFO - Resampling mask to target shape and spacing
2025-10-27 14:13:29,615 - root - INFO - Resize Spacing: [tensor(1.4824, dtype=torch.float64), tensor(1.4824, dtype=torch.float64), tensor(1.5586, dtype=torch.float64)] -> [1.5, 1.5, 1.5]
2025-10-27 14:13:29,616 - root - INFO - Output size: [256, 256, 256] -> [256, 256, 256]
2025-10-27 14:13:30,496 - root - INFO - Resampling mask to target shape and spacing
2025-10-27 14:13:30,498 - root - INFO - Resize Spacing: [tensor(1.4824, dtype=torch.float64), tensor(1.4824, dtype=torch.float64), tensor(1.3652, dtype=torch.float64)] -> [1.5, 1.5, 1.5]
2025-10-27 

100%|██████████| 30/30 [00:05<00:00,  5.62it/s]

2025-10-27 14:13:38,766 - root - INFO - ---- Latent features generation time: 5.372609615325928 seconds ----
2025-10-27 14:13:38,792 - root - INFO - ---- Start decoding latent features into images... ----


2025-10-27 14:13:40,594 - root - INFO - ---- Image decoding time: 1.8011527061462402 seconds ----
2025-10-27 14:13:42,143 INFO image_writer.py:197 - writing: /localhome/local-tranminhq/datasets/maisi/maisi_ct_generative/output/sample_20251027_141342_133236_image.nii.gz
2025-10-27 14:13:43,432 INFO image_writer.py:197 - writing: /localhome/local-tranminhq/datasets/maisi/maisi_ct_generative/output/sample_20251027_141342_133236_label.nii.gz


[None,
 [['/localhome/local-tranminhq/datasets/maisi/maisi_ct_generative/output/sample_20251027_141342_133236_image.nii.gz',
   '/localhome/local-tranminhq/datasets/maisi/maisi_ct_generative/output/sample_20251027_141342_133236_label.nii.gz']]]